In [1]:
import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations}
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from pandas import Series
from pandas import DataFrame
import timeit
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import timeit
from sklearn.metrics.pairwise import cosine_similarity
import math

In [247]:
tag_table = pd.read_table("C:/R_data_new/delicious/tags.dat",dtype={"id":"str","value":"str"})

In [248]:
item_table = pd.read_table("C:/R_data_new/delicious/bookmarks.dat",dtype={"id":"str","value":"str"})

In [249]:
item_table_arrange = item_table[["id","title"]]

In [250]:
item_id_title_dic = item_table_arrange.set_index("id")["title"].to_dict()

In [251]:
tag_id_title_dic =  tag_table.set_index("id")["value"].to_dict()

In [2]:
user_item_tag_table = pd.read_table("C:/R_data_new/delicious/user_taggedbookmarks.dat",dtype={"userID":"str","bookmarkID":"str","tagID":"str"})

In [3]:
user_item_tag_table = user_item_tag_table[user_item_tag_table.columns[:3]]

In [4]:
user_item_tag_table_test = user_item_tag_table

In [5]:
user_item_tag_table_test.dtypes

userID        object
bookmarkID    object
tagID         object
dtype: object

In [6]:
user_item_tag_table.shape

(437593, 3)

In [267]:
user_item_tag_table_test.head(40)

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,1
3,8,7,6
4,8,7,7
5,8,8,1
6,8,8,8
7,8,8,9
8,8,9,1
9,8,9,10


In [226]:
user_tags = {}
tag_items = {}
item_tags = {}
users = []
tags = []
items = []
n=0
for a in user_item_tag_table_test.index:
    userID = user_item_tag_table_test.ix[a,["userID"]].values[0]
    tagID = user_item_tag_table_test.ix[a,["tagID"]].values[0]
    itemID = user_item_tag_table_test.ix[a,["bookmarkID"]].values[0]
    if userID not in users:
        users.append(str(userID))
    
    if tagID not in tags:
        tags.append(str(tagID))
    
    if itemID not in items:
        items.append(str(itemID))
    
    user_tags.setdefault(str(userID),{})
    user_tags[str(userID)].setdefault(str(tagID),0)
    user_tags[str(userID)][str(tagID)]+=1
    
    tag_items.setdefault(str(tagID),{})
    tag_items[str(tagID)].setdefault(str(itemID),0)
    tag_items[str(tagID)][str(itemID)]+=1
    
    item_tags.setdefault(str(itemID),{})
    item_tags[str(itemID)].setdefault(str(tagID),0)
    item_tags[str(itemID)][str(tagID)]+=1
    
    
    n+=1
    if n%10000==0:
        print n

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000


Do not remove the tagged items for recommendations

general algorithmn

In [9]:
def Recommend(user):
    recommend_items = {}
    for tag, wut in user_tags[user].items():
        if tag not in tag_items.keys():
            continue
        else:
            for item, wti in tag_items[tag].items():
                if item not in recommend_items:
                    recommend_items[item] = wut*wti
                else:
                    recommend_items[item] += wut*wti
    return recommend_items 

In [214]:
def RecommendTable(user):
    recommend_items = {}
    items_title = {}
    for tag, wut in user_tags[user].items():
        if tag not in tag_items.keys():
            continue
        else:
            for item, wti in tag_items[tag].items():
                if item not in recommend_items:
                    recommend_items[item] = wut*wti
                    items_title[item] = item_id_title_dic[item]
                else:
                    recommend_items[item] += wut*wti
                    
    
    data_series= pd.Series(recommend_items)
    data_frame = pd.DataFrame(data_series)
    data_frame = data_frame.rename(columns={0:"weight"})
    
    data_series1= pd.Series(items_title)
    data_frame1 = pd.DataFrame(data_series1)
    data_frame1 = data_frame1.rename(columns={0:"title"})
    
    dataframe_overall = pd.concat([data_frame, data_frame1], axis=1)
    dataframe_overall = dataframe_overall.sort_values(by = ["weight"],ascending = False)
    dataframe_overall = dataframe_overall.reset_index()
    dataframe_overall = dataframe_overall.rename(columns={"index":"itemID"})
    return dataframe_overall

Tags similarity calculation

In [281]:
def SimilarityScoreTags(tag_items,i,j):
    ret = 0
    for b,wib in tag_items[i].items():
        if b in tag_items[j]:
            ret+= wib *tag_items[j][b]
    ni=0
    nj=0
    for b,w in tag_items[i].items():
        ni+=w*w
    for b,w in tag_items[j].items():
        nj+=w*w
    
    if ret ==0:
        return 0
    return ret / math.sqrt(ni * nj) 

In [216]:
def FindUserTag(user):
    user_tags_name_dic = {}
    user_tags_score_dic = user_tags[user]
    
    
    
    for key in user_tags_score_dic.keys():
        tag_title = tag_id_title_dic[key]
        user_tags_name_dic[key] = tag_title
    
    data_series= pd.Series(user_tags_score_dic)
    data_frame = pd.DataFrame(data_series)
    data_frame = data_frame.rename(columns={0:"weight"})
    
    data_series1= pd.Series(user_tags_name_dic)
    data_frame1 = pd.DataFrame(data_series1)
    data_frame1 = data_frame1.rename(columns={0:"title"})
    
    dataframe_overall = pd.concat([data_frame, data_frame1], axis=1)
    dataframe_overall = dataframe_overall.sort_values(by = ["weight"],ascending = False)
    dataframe_overall = dataframe_overall.reset_index()
    dataframe_overall = dataframe_overall.rename(columns={"index":"tagID"})
    return dataframe_overall

In [259]:
def FindItemTag(item):
    item_tags_name_dic = {}
    item_tags_score_dic = item_tags[item]
    for key in item_tags_score_dic.keys():
        tag_name = tag_id_title_dic[key]
        item_tags_name_dic[key] = tag_name
    
    data_series= pd.Series(item_tags_score_dic)
    data_frame = pd.DataFrame(data_series)
    data_frame = data_frame.rename(columns={0:"weight"})
    
    data_series1= pd.Series(item_tags_name_dic)
    data_frame1 = pd.DataFrame(data_series1)
    data_frame1 = data_frame1.rename(columns={0:"title"})
    
    dataframe_overall = pd.concat([data_frame, data_frame1], axis=1)
    dataframe_overall = dataframe_overall.sort_values(by = ["weight"],ascending = False)
    dataframe_overall = dataframe_overall.reset_index()
    dataframe_overall = dataframe_overall.rename(columns={"index":"itemID"})
    return dataframe_overall

In [277]:
RecommendTable("259")

,itemID,weight,title
0,5448,398,HTML5 Boilerplate - A rock-solid default for H...
1,10043,282,Modernizr
2,8981,244,Periodic Table of the Elements - Josh Duck
3,4041,207,Ultimate CSS Gradient Generator - ColorZilla.com
4,8294,193,HTML5Rocks - Home
5,11115,192,Polymaps
6,2720,182,Prezi - The zooming presentation editor
7,6123,182,41Latitude - Styled Maps Using Google Maps API...
8,9320,182,zen-coding - Project Hosting on Google Code
9,589,174,The Wilderness Downtown


In [278]:
FindUserTag("259")

,tagID,weight,title
0,859,15,ruby
1,199,10,tools
2,860,9,rubyonrails
3,431,9,css
4,878,8,javascript
5,159,6,google
6,861,5,flash
7,894,5,rails3
8,143,5,video
9,455,5,html5


In [279]:
FindItemTag("5448")

,itemID,weight,title
0,455,23,html5
1,456,11,html
2,431,11,css
3,1071,10,template
4,878,8,javascript
5,2185,7,templates
6,1063,7,webdev
7,4,7,development
8,4271,6,boilerplate
9,432,6,framework


In [284]:
SimilarityScoreTags(tag_items,"455","431")

0.2673188835864117